# ERS NIRCam Demo with Eureka!


#### Authors: Megan Mansfield and Eva-Maria Ahrer
 Hello! This notebook shows a full demonstration reducing and analyzing the NIRCam/F322W2 transit of WASP-39b from the JWST Transiting Exoplanet Early Release Science Program (ERS 1366). Following this pipeline, you should be able to reproduce the NIRCam spectrum presented in the main text of Ahrer et al. (2022). 
 
This data reduction uses the [Eureka!](https://github.com/kevin218/Eureka) pipeline. Before running this Jupyter Notebook, please follow the steps in "README.md" to set up an environment for this demonstration and install Eureka. Note that this demonstration is using a fixed version of Eureka! to ensure that no future updates break the example here, so even if you have the current version of Eureka! installed on your own machine you'll need to follow the set-up steps to ensure this demo works properly. For more information on Eureka! see its documentation or refer to Bell et al. (2022).

### First, let's import packages!

In [ ]:
import sys
sys.path.append('./')
sys.path.insert(0,'./')
import eureka.S1_detector_processing.s1_process as s1
import eureka.S2_calibrations.s2_calibrate as s2
import eureka.S3_data_reduction.s3_reduce as s3
import eureka.S4_generate_lightcurves.s4_genLC as s4
import eureka.S5_lightcurve_fitting.s5_fit as s5
import eureka.S6_planet_spectra.s6_spectra as s6
from astroquery.mast import Observations
import eureka.lib.mastDownload as md
from astropy.io import ascii

from eureka.lib import readECF

eventlabel = 'nircam_wasp39b'
ecf_path = './ecf/'

### Next, download the data set.
#### Data can be downloaded from MAST using Eureka!. Here we will go through important keywords to pull the correct data for this tutorial.
1. proposal_id, observation, visit - All three are used to specify the exact observation you want to download for a given program. The proposal_id can be found by searching for the program from the list of accepted programs at https://www.stsci.edu/jwst/science-execution/approved-programs. After finding the correct program, click "Visit Status Information" to find the correct observation and visit numbers for the data set you'd like to download.

2. calib_level, subgroup - These specify which level in the JWST pipeline you'd like to retrieve data products at and which data product you will get. calib_level=0 is raw, which you'll likely never want to download. calib_level=1, subgroup='UNCAL' downloads uncalibrated data and is the standard choice for this tutorial. However, you can also download data at level 2 and start from a later point in this notebook. If you set calib_level=2, subgroup='RATEINTS', skip Stage 1 below and start at Stage 2. If you set calib_level=2, subgroup='CALINTS', skip Stages 1 and 2 below and start at Stage 3. Note that if you do this you may get different results than in the paper because you'll be relying on the STScI JWST pipeline for Stages 1/2 instead of using Eureka!.

3. download_dir - This is just a temporary storage location as the data is being gathered from MAST.

4. final_dir - This is the final location where the downloaded data files will be stored. For this tutorial, since we're starting with uncalibrated data, we're placing it in a directory called "./Uncalibrated". You can name the directory anything you want as long as you provide the right directory name to Eureka! later in the tutorial.


In [ ]:
proposal_id = '01366'
observation = 2
visit = 1

calib_level = [1]
subgroup = 'UNCAL'

download_dir = './wasp39b'
final_dir = './Uncalibrated/'

# Apply standard filters to identify files for download
table = md.filterJWST(proposal_id, observation, visit, calib_level, subgroup)

# Download data products, returns manifest of files downloaded.
manifest = Observations.download_products(table, curl_flag=False,
                                          download_dir=download_dir)

# Consolidate and move data into new directory
md.consolidate(manifest, final_dir)

# Delete empty temporary directory structure
md.cleanup(download_dir)

#### Note: Eureka! is divided into six "Stages". Each Stage has a corresponding Eureka! Control File or .ecf file, and in Stage 5 there's also a Eureka! Parameter File or .epf. 
These files are named "S2_eventlabel.ecf" where the first half of the name (S2, S3, etc.) refers to the Stage that .ecf file interacts with, and the second half refers to the "eventlabel" keyword we defined above. See the Eureka! documentation for a full description of each stage and all keywords in the .ecf files. Here we'll give a very brief summary of each Stage and discuss a few important keywords for reproducing the paper results.

# Stage 1: Correcting detector-level effects and fitting the up-the-ramp slope.
#### Most important keywords in the .ecf:
1. jump_rejection_threshold - this sets the sigma threshold for rejecting a jump in the up-the-ramp slope as due to a cosmic ray hit. The standard value in the jwst pipeline is 4.0, but this data reduction found a value of 6.0 produced better results.
2. topdir - Edit this to make it the path to where you've downloaded the data for this demo. The path should look like this: /path/to/NIRCam_demo/wasp39b_data/
3. inputdir - This keyword tells Eureka! where to look for the Uncalibrated outputs to feed into the Stage 1 code. This path is relative to "topdir". In this demo, we set inputdir=/Uncalibrated
4. outputdir - This is where Eureka! will save all the Stage 2 outputs, including plots and log files. Here we set outputdir=/Stage1, which means the output files will be saved within a folder on the path /path/to/NIRCam_demo/NIRCam_full_data/Stage1/
5. suffix - This is used to identify which subset of files from the ones downloaded from MAST will be used. For this data set, the long wavelength spectra are contained in files with the naming pattern '04103_\*nrcalong_uncal'
#### Note that topdir, inputdir, and outputdir work the same in every .ecf file, so in each file edit them so that "inputdir" points to the previous Stage's outputs, and "outputdir" points to where you want to save that Stage's outputs.

In [ ]:
ecffile = 'S1_' + eventlabel + '.ecf'
input_meta_S1 = readECF.MetaClass(ecf_path, ecffile)

input_meta_S1.jump_rejection_threshold = 6.0
input_meta_S1.topdir = '/storage/astro2/phrgmk/Data/JWST/NIRCam/WASP-39/'
input_meta_S1.inputdir = input_meta_S1.topdir + '/Uncalibrated/'
input_meta_S1.outputdir = input_meta_S1.topdir + '/Stage1'
input_meta_S1.suffix = '04103_*nrcalong_uncal'

### And that's it! Let's run Stage 1!

In [ ]:
s1_meta = s1.rampfitJWST(eventlabel, ecf_path=ecf_path, input_meta=input_meta_S1) 

# Stage 2: Additional pre-spectral-extraction steps, such as assignment of the world coordinate system, flat fielding, wavelength calibration. Outputs calibrated 2D images.

#### If you have downloaded the rateints.fits files start here!

#### Most important keywords in the .ecf:
1. skip_bkg_subtract - Background subtraction is skipped by default for time-series observations such as exoplanet transits.
2. hide_plots - This keyword will close plots as they are created. It is very useful for avoiding memory issues if you are printing a large number of plots, so we recommend leaving it set to True.


In [ ]:
ecffile = 'S2_' + eventlabel + '.ecf'
input_meta_S2 = readECF.MetaClass(ecf_path, ecffile)
input_meta_S2.skip_bkg_subtract = True
input_meta_S2.hide_plots = True

#### If start running this notebook at this stage (using rateints.fits outputs), then you need to uncomment the lines in the cell below and define the topdir and inputdir parameters here

In [ ]:
#input_meta_S2.topdir = '/data/JWST/wasp39b/nircam_f322w2/'
#input_meta_S2.inputdir = 'S1/nrcalong/'

### Let's run Stage 2!

In [ ]:
s2_meta = s2.calibrateJWST(eventlabel, ecf_path=ecf_path, s1_meta=s1_meta, input_meta=input_meta_S2)

# Stage 3: Identify source position, perform background subtraction, perform spectral extraction to produce time series of 1D extracted spectra.

#### If you have downloaded the calints.fits files start here!

#### Most important keywords in the .ecf:
1. ncpu - Number of CPUs on your machine. This won't affect the data reduction except to make it run faster if you are able to run it on more CPUs. You can change this number to fit whatever machine you're running on.
2. ywindow and xwindow - These specify the region of the image that you're interested in performing background subtraction and spectral extraction on. Note that this is the *full* image, not just the window surrounding the spectral trace. For this demo, the ywindow is set to ignore reference pixels on the edges of the detector, and the xwindow is set to select the region of higher throughput where the spectral trace can actually be seen in the image. These values were selected based on viewing an image in ds9 or another fits file viewer and looking at the position of the spectral trace.

#### The next several parameters have to do with the background subtraction. Eureka! performs background subtraction by identifying the spectral trace, masking out a region surrounding the spectral trace, and using the remaining pixels as the background.

3. bg_hw - Defines the half-width of the masked area not included in background subtraction. In this example, a value of 14 means that 14 pixels both above and below the identified source position are excluded from each column. We tested a few different values and found that 14 minimized the MAD of the resulting light curves, as it was large enough to exclude contamination from the spectrum but not so large that the background subtraction was affected by having less pixels to estimate the background from.
4. bg_deg - The background is subtracted by doing a column-by-column polynomial fit, and bg_deg defines the degree of that fit. Setting bg_deg = -1 will just calculate and subtract out a median for each column. For this reduction, bg_deg=1 removed the background sufficiently well.

#### Now some parameters for how we'll do the spectral extraction!
5. spec_hw - Defines the half-width of the region you extract the spectrum from. In this example, a value of 9 means that the extraction will be perfomed on a box of pixels extending 9 up and 9 down from the identified source position. We tested several values and found that a half-width of 9 minimized the MAD of the resulting light curves, as it was wide enough to catch the edges of the spectrum but not so wide that it added too much background contamination.

#### Finally, some parameters for printing diagnostics and saving output.
6. isplots_S3 - How many plots do you want to create? This can be set to 1, 3, or 5, where a bigger number will print out more different types of diagnostic plots. The default in this demo is 5 so that you can see all the diagnostics.

In [ ]:
ecffile = 'S3_' + eventlabel + '.ecf'
input_meta_S3 = readECF.MetaClass(ecf_path, ecffile)

input_meta_S3.ncpu = 1
input_meta_S3.xwindow = [4,64]
input_meta_S3.ywindow = [4,1704]

input_meta_S3.bg_hw = 14
input_meta_S3.bg_deg = 1

input_meta_S3.spec_hw = 9

input_meta_S3.isplots_S3 = 3
input_meta_S3.hide_plots = True


#### If you started with the Stage 2 output files (calints.fits), then you need to uncomment the lines in the cell below and define the topdir and inputdir parameters here.

In [ ]:
#input_meta_S3.topdir = '/data/JWST/wasp39b/nircam_f322w2'
#input_meta_S3.inputdir = 'S2/nrcalong/'

In [ ]:
s3_spec, s3_meta = s3.reduce(eventlabel, ecf_path=ecf_path, s2_meta=s2_meta, input_meta=input_meta_S3)

# Stage 4: Convert 1D extracted spectra to time series light curves

#### Most important keywords in the .ecf:
1. nspecchan - number of spectroscopic channels
2. compute_white - whether to compute the white-light lightcurve
3. wave_min and wave_max - the wavelength range in micron
4. clip_unbinned and clip_binned - whether to perform sigma-clipping on the unbinned and/or binned 1D time series
5. sigma - the number of sigmas a point must be from the rolling median to be considered an outlier

#### Limb-darkening coefficients using exotic-ld (https://exotic-ld.readthedocs.io/en/latest/)
6. compute_ld - whether to compute the limb-darkening coefficients with exotic-ld
7. exotic_ld_direc - directory for ancillary files for exotic-ld (see documentation of exotic-ld)
8. exotic_ld_grid - whether to use 3D or 1D stellar model grids
9. exotic_ld_file - if you want to use custom throughput file (leave empty if using default)

In [ ]:
ecffile = 'S4_' + eventlabel + '.ecf'
input_meta_S4 = readECF.MetaClass(ecf_path, ecffile)


input_meta_S4.nspecchan = 110
input_meta_S4.wave_min = 2.405
input_meta_S4.wave_max = 4.055

input_meta_S4.compute_white = True 

input_meta_S4.clip_unbinned = False   
input_meta_S4.clip_binned = True    
input_meta_S4.sigma = 4     

# Limb-darkening parameters needed to compute exotic-ld
input_meta_S4.compute_ld = False
input_meta_S4.exotic_ld_direc = '/data/exotic-ld_data/' 
input_meta_S4.exotic_ld_grid = '3D' 
input_meta_S4.exotic_ld_file = '/data/exotic-ld_data/NIRCam_throughput_full.csv' 


In [ ]:
s4_spec, s4_lc, s4_meta = s4.genlc(eventlabel, ecf_path=ecf_path, s3_meta=s3_meta, input_meta=input_meta_S4)

# Stage 5: Light Curve Fitting

#### Most important keywords in the .ecf:
1. fit_method - Which fitting method to use, options are: lsq, emcee, dynesty (can list multiple types separated by commas)
2. run_myfuncs - What does the fit consist of? Options are: batman_tr (transit model), batman_ecl (eclipse model), sinusoid_pc (phase curve), expramp (exponential ramp), polynomial, step, and GP (Gaussian Process). Must list all models you want to use!
3. use_generate_ld - Whether to use the 'exotic-ld' limb-darkening coefficients generated in Stage 4

In [ ]:
ecffile = 'S5_' + eventlabel + '.ecf'
input_meta_S5 = readECF.MetaClass(ecf_path, ecffile)

input_meta_S5.fit_method = '[emcee]'              
input_meta_S5.run_myfuncs = '[batman_tr,polynomial]' 

input_meta_S5.use_generate_ld = 'exotic-ld' 

#### Specific fitter parameters are in the cell below
Least-squares (lsq)
1. lsq_method - the scipy.optimize.minimize optimization method to use

Markov Chain Monte Carlo (MCMC)
1. lsq_first - whether to run a least-squares fit first and use as initial values for MCMC fit
2. run_steps - number of steps
3. run_nwalkers - number of walkers
4. run_nburn - number of run_nsteps should be discarded as burn-in steps

Dynesty (nested sampling)
1. run_nlive - number of live points
2. run_tol - tolerance value i.e. convergence criterion

In [ ]:
#fitter parameters
#lsq
input_meta_S5.lsq_method = 'Powell'

#mcmc
input_meta_S5.lsq_first = True    
input_meta_S5.run_nsteps = 500
input_meta_S5.run_nwalkers = 100
input_meta_S5.run_nburn = 100     

#dynesty
input_meta_S5.run_nlive = 1024    
input_meta_S5.run_tol = 0.1


## Important: Check your Parameter file (.epf file) for all parameter specific inputs e.g. prior types and ranges



### Let's fit some light curves!

In [ ]:
s5_meta = s5.fitlc(eventlabel, ecf_path=ecf_path, s4_meta=s4_meta, input_meta=input_meta_S5)

# Stage 6: Create Final Spectrum

#### Most important keywords in the .ecf:
1. y_unit - For plotting the transmission spectrum, options include Rp/Rs, (Rp/Rs)^2, Fp/Fs
2. y_sclar - Can be used to convert to ppm (1e6), percent (100), etc.

In [ ]:
ecffile = 'S6_' + eventlabel + '.ecf'
input_meta_S6 = readECF.MetaClass(ecf_path, ecffile)

input_meta_S6.y_unit = '(Rp/Rs)^2' 
input_meta_S6.y_scalar = 100 

### Let's run the final stage to get a transmission spectrum!

In [ ]:
s6_meta = s6.plot_spectra(eventlabel, ecf_path=ecf_path, s5_meta=s5_meta, input_meta=input_meta_S6)